In [1]:
import cv2
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from ultralytics import YOLO
import threading
import time
from datetime import datetime
from PIL import Image, ImageTk

class SimpleTrafficAnalyzer:
    def __init__(self):
        self.model_people = YOLO(r"D:\Programm_git\Traffic_analysis\learned\train9\weights\best.pt")
        self.model_cars = YOLO(r"D:\Programm_git\Traffic_analysis\learned\train12\weights\best.pt")
        
        self.unique_people_ids = set()
        self.unique_car_ids = set()
        self.interval_data = []
        
        self.is_processing = False
        self.video_path = None
        self.cap = None
        
        self.setup_ui()
    
    def setup_ui(self):
        self.root = tk.Tk()
        self.root.title("Анализ трафика")
        self.root.geometry("900x600")
        
        main_frame = ttk.Frame(self.root, padding="10")
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        control_frame = ttk.Frame(main_frame)
        control_frame.pack(fill=tk.X, pady=5)
        
        ttk.Button(control_frame, text="Выбрать видео", command=self.select_video).pack(side=tk.LEFT, padx=5)
        self.start_btn = ttk.Button(control_frame, text="Начать анализ", command=self.start_analysis)
        self.start_btn.pack(side=tk.LEFT, padx=5)
        
        self.file_label = ttk.Label(control_frame, text="Файл не выбран")
        self.file_label.pack(side=tk.LEFT, padx=20)
        

        video_frame = ttk.LabelFrame(main_frame, text="Видео с трекингом", padding="5")
        video_frame.pack(fill=tk.BOTH, expand=True, pady=10)
        
        self.video_label = ttk.Label(video_frame, text="Видео будет отображаться здесь", 
                                    background='black', foreground='white')
        self.video_label.pack(fill=tk.BOTH, expand=True)

        bottom_frame = ttk.Frame(main_frame)
        bottom_frame.pack(fill=tk.X, pady=5)
        
        self.progress = ttk.Progressbar(bottom_frame, mode='determinate')
        self.progress.pack(fill=tk.X, pady=5)
        
        self.status_label = ttk.Label(bottom_frame, text="Готов к работе")
        self.status_label.pack(anchor=tk.W)
    
    def select_video(self):
        filename = filedialog.askopenfilename(
            filetypes=[("Video files", "*.mp4 *.avi *.mov *.mkv"), ("All files", "*.*")]
        )
        if filename:
            self.video_path = filename
            self.file_label.config(text=filename.split('/')[-1])
    
    def start_analysis(self):
        if not self.video_path:
            messagebox.showerror("Ошибка", "Сначала выберите видео файл")
            return
            
        self.is_processing = True
        self.unique_people_ids.clear()
        self.unique_car_ids.clear()
        self.interval_data.clear()
        
        self.start_btn.config(state=tk.DISABLED)
        
        thread = threading.Thread(target=self.process_video)
        thread.daemon = True
        thread.start()
    
    def process_video(self):
        self.cap = cv2.VideoCapture(self.video_path)
        
        if not self.cap.isOpened():
            self.show_error("Не удалось открыть видео")
            return
        
        fps = self.cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        duration = total_frames / fps
        
        interval_seconds = 5 * 60 
        interval_frames = int(interval_seconds * fps)
        
        frame_count = 0
        last_interval_end = 0
        current_interval = 1
        while self.is_processing:
            ret, frame = self.cap.read()
            if not ret:
                break

            results_people = self.model_people.track(frame, persist=True, tracker="bytetrack.yaml", verbose=False)
            results_cars = self.model_cars.track(frame, persist=True, tracker="bytetrack.yaml", verbose=False)


            if results_people and results_people[0].boxes.id is not None:
                for track_id in results_people[0].boxes.id.int().tolist():
                    self.unique_people_ids.add(track_id)
            
            if results_cars and results_cars[0].boxes.id is not None:
                for track_id in results_cars[0].boxes.id.int().tolist():
                    self.unique_car_ids.add(track_id)
            current_time = frame_count / fps

            if current_time >= last_interval_end + interval_seconds:
                interval_data = {
                    'interval': current_interval,
                    'time_minutes': current_time / 60,
                    'people_count': len(self.unique_people_ids),
                    'cars_count': len(self.unique_car_ids),
                    'timestamp': datetime.now()
                }
                self.interval_data.append(interval_data)
                last_interval_end = current_time
                current_interval += 1


            annotated_frame = frame.copy()
            if results_people:
                annotated_frame = results_people[0].plot(img=annotated_frame)
            if results_cars:
                annotated_frame = results_cars[0].plot(img=annotated_frame)


            cv2.putText(annotated_frame, f"People: {len(self.unique_people_ids)}", (30, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(annotated_frame, f"Cars: {len(self.unique_car_ids)}", (30, 100),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
            cv2.putText(annotated_frame, f"Time: {current_time/60:.1f}m", (30, 150),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            self.display_frame(annotated_frame)
            
            progress = (frame_count / total_frames) * 100
            self.update_progress(progress, frame_count, total_frames)
            frame_count += 1
            time.sleep(0.03)
        
        final_data = {
            'interval': 'FINAL',
            'time_minutes': duration / 60,
            'people_count': len(self.unique_people_ids),
            'cars_count': len(self.unique_car_ids),
            'timestamp': datetime.now()
        }
        self.interval_data.append(final_data)
        
        if self.cap:
            self.cap.release()
        self.is_processing = False
        self.show_final_report()
        self.root.after(0, lambda: self.start_btn.config(state=tk.NORMAL))
    
    def display_frame(self, frame):
        """Отображает кадр в интерфейсе"""
        try:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            h, w = frame.shape[:2]
            
            max_width = 850
            max_height = 450
            
            if w > max_width or h > max_height:
                scale = min(max_width/w, max_height/h)
                new_w = int(w * scale)
                new_h = int(h * scale)
                frame_resized = cv2.resize(frame_rgb, (new_w, new_h))
            else:
                frame_resized = frame_rgb
            
            img = Image.fromarray(frame_resized)
            imgtk = ImageTk.PhotoImage(image=img)
            
            self.video_label.configure(image=imgtk)
            self.video_label.image = imgtk
            
        except Exception as e:
            print(f"Ошибка отображения кадра: {e}")
    
    def update_progress(self, progress, current_frame, total_frames):
        def update():
            self.progress['value'] = progress
            self.status_label.config(
                text=f"Обработано: {current_frame}/{total_frames} кадров ({progress:.1f}%)"
            )
        self.root.after(0, update)
    
    def show_final_report(self):
        if not self.interval_data:
            return
        
        final_data = self.interval_data[-1]
        
        report_window = tk.Toplevel(self.root)
        report_window.title("Итоговый отчет")
        report_window.geometry("500x400")
        
        text_frame = ttk.Frame(report_window, padding="10")
        text_frame.pack(fill=tk.BOTH, expand=True)
        
        report_text = tk.Text(text_frame, wrap=tk.WORD, font=('Arial', 10))
        scrollbar = ttk.Scrollbar(text_frame, orient=tk.VERTICAL, command=report_text.yview)
        report_text.configure(yscrollcommand=scrollbar.set)
        
        report_text.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        report_content = "ОТЧЕТ АНАЛИЗА ТРАФИКА\n"
        report_content += "=" * 40 + "\n\n"
        
        report_content += f"Видео файл: {self.video_path}\n"
        report_content += f"Дата анализа: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n"
        
        report_content += f"Общее количество людей: {final_data['people_count']}\n"
        report_content += f"Общее количество машин: {final_data['cars_count']}\n"
        report_content += f"Общее время видео: {final_data['time_minutes']:.2f} минут\n\n"
        
        if len(self.interval_data) > 1:
            report_content += "Данные по 5-минутным интервалам:\n"
            report_content += "-" * 35 + "\n"
            for data in self.interval_data[:-1]:
                report_content += (f"Интервал {data['interval']} ({data['time_minutes']:.1f} мин): "
                                f"Люди: {data['people_count']}, Машины: {data['cars_count']}\n")
        
        report_text.insert(tk.END, report_content)
        report_text.config(state=tk.DISABLED)
        save_frame = ttk.Frame(report_window)
        save_frame.pack(fill=tk.X, padx=10, pady=5)
        
        ttk.Button(save_frame, text="Сохранить отчет", 
                    command=lambda: self.save_results_to_file(report_content)).pack(side=tk.RIGHT)
        
        self.save_results_to_file(report_content)
    
    def save_results_to_file(self, report_content=None):
        if not self.interval_data:
            return
            
        filename = f"traffic_analysis_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
        
        try:
            if report_content is None:
                final_data = self.interval_data[-1]
                report_content = "ОТЧЕТ АНАЛИЗА ТРАФИКА\n" + "="*40 + "\n\n"
                report_content += f"Видео файл: {self.video_path}\n"
                report_content += f"Дата анализа: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n"
                report_content += f"Общее количество людей: {final_data['people_count']}\n"
                report_content += f"Общее количество машин: {final_data['cars_count']}\n"
                report_content += f"Общее время видео: {final_data['time_minutes']:.2f} минут\n\n"
                
                if len(self.interval_data) > 1:
                    report_content += "Данные по 5-минутным интервалам:\n" + "-"*35 + "\n"
                    for data in self.interval_data[:-1]:
                        report_content += (f"Интервал {data['interval']} ({data['time_minutes']:.1f} мин): "
                                        f"Люди: {data['people_count']}, Машины: {data['cars_count']}\n")
            
            with open(filename, 'w', encoding='utf-8') as f:
                f.write(report_content)
            
            messagebox.showinfo("Сохранено", f"Отчет сохранен в файл: {filename}")
        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось сохранить отчет: {e}")
    
    def show_error(self, message):
        def show():
            messagebox.showerror("Ошибка", message)
            self.start_btn.config(state=tk.NORMAL)
        self.root.after(0, show)
    
    def on_closing(self):
        self.is_processing = False
        if self.cap:
            self.cap.release()
        self.root.destroy()
    
    def run(self):
        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)
        self.root.mainloop()

if __name__ == "__main__":
    app = SimpleTrafficAnalyzer()
    app.run()